# TorchMD API tutorial

## System setup

We use the `moleculekit` library for reading the input topologies and starting coordinates

In [1]:
from moleculekit.molecule import Molecule
import os

testdir = "./data/"
mol = Molecule(os.path.join(testdir, "backbone-no-improp.psf"))  # Reading the system topology
mol.read(os.path.join(testdir, "backbone.pdb"))  # Reading the initial simulation coordinates
#mol.read(os.path.join(testdir, "input.xsc"))  # Reading the box dimensions

In [2]:

mydict = {'terms': {'term1': {'phi_k': 0.0, 'per': 1, 'phase': 0.0}, 'term2': {'phi_k': 3.0, 'per': 1, 'phase': 0.0}}}
print(mydict['terms']["term1"]['phi_k'])
for term in mydict['terms']:
    print(term)
    print(mydict["terms"][term]['phi_k'])

0.0
term1
0.0
term2
3.0


Next we will load a forcefield file and use the above topology to extract the relevant parameters which will be used for the simulation

In [8]:
from torchmd.forcefields.forcefield import ForceField
from torchmd.parameters import Parameters
import torch

precision = torch.float
device = "cuda:0"

ff = ForceField.create(mol, os.path.join(testdir, "param_bb-3.0.yaml"))
#parameters = Parameters(ff, mol, precision=precision, device=device)
parameters = Parameters(ff, mol, precision=precision)

{'terms': {'term1': {'phi_k': 0.0, 'per': 1, 'phase': 0.0}}}
term1
{'terms': {'term1': {'phi_k': 0.4, 'per': 1, 'phase': 0.0}}}
term1
{'terms': {'term1': {'phi_k': 1.6, 'per': 1, 'phase': 0.0}}}
term1
{'terms': {'term1': {'phi_k': 0.2, 'per': 1, 'phase': 180.0}}}
term1
{'terms': {'term1': {'phi_k': 2.5, 'per': 2, 'phase': 180.0}}}
term1
{'terms': {'term1': {'phi_k': 0.0, 'per': 1, 'phase': 0.0}}}
term1
{'terms': {'term1': {'phi_k': 0.4, 'per': 1, 'phase': 0.0}}}
term1
{'terms': {'term1': {'phi_k': 1.6, 'per': 1, 'phase': 0.0}}}
term1
{'terms': {'term1': {'phi_k': 0.2, 'per': 1, 'phase': 180.0}}}
term1
{'terms': {'term1': {'phi_k': 2.5, 'per': 2, 'phase': 180.0}}}
term1
{'terms': {'term1': {'phi_k': 0.0, 'per': 1, 'phase': 0.0}}}
term1
{'terms': {'term1': {'phi_k': 0.4, 'per': 1, 'phase': 0.0}}}
term1
{'terms': {'term1': {'phi_k': 1.6, 'per': 1, 'phase': 0.0}}}
term1
{'terms': {'term1': {'phi_k': 0.2, 'per': 1, 'phase': 180.0}}}
term1
{'terms': {'term1': {'phi_k': 2.5, 'per': 2, 'phase'

Now we can create a `System` object which will contain the state of the system during the simulation, including:
1. The current atom coordinates
1. The current box size
1. The current atom velocities
1. The current atom forces

In [9]:
from torchmd.integrator import maxwell_boltzmann
from torchmd.systems import System

system = System(mol.numAtoms, nreplicas=1, precision=precision, device=None)
system.set_positions(mol.coords)
#system.set_box(mol.box)
#system.set_velocities(maxwell_boltzmann(parameters.masses, T=300, replicas=1))

Lastly we will create a `Force` object which will be used to evaluate the potential on a given `System` state

In [10]:
from torchmd.forces import Forces

forces = Forces(parameters, cutoff=9, rfa=True, switch_dist=7.5)
# Evaluate current energy and forces. Forces are modified in-place
Epot = forces.compute(system.pos, system.box, system.forces, returnDetails=True)

print(Epot)
print(system.forces)

[{'electrostatics': 627.0636596679688, 'lj': 11.866900444030762, 'bonds': 20.15338134765625, 'angles': 21.72222900390625, 'dihedrals': 7.024267196655273, '1-4': 0.0, 'impropers': 0.0, 'external': 0.0}]
tensor([[[ 9.4370e+00, -4.1128e+01, -1.6852e+01],
         [-4.1977e+00,  3.3838e+01,  1.6829e+01],
         [-1.9985e+01, -7.5960e-01, -2.3018e+01],
         [ 2.0118e+01,  1.3214e+00,  1.3461e+01],
         [-5.5166e+00,  1.1977e+00,  3.8847e-01],
         [ 2.1103e+00,  2.6962e+00,  1.7270e+01],
         [-1.0189e+01,  1.5651e+01, -3.7272e+01],
         [ 4.3254e+00, -1.3596e+01,  6.9031e+00],
         [ 1.1188e+01, -3.6457e+00,  8.7871e+00],
         [-4.8800e+00,  7.3912e-02,  1.7853e+00],
         [-3.7675e+01,  1.7862e+01,  3.7025e+01],
         [ 3.4382e+01, -2.1014e+01, -1.6628e+01],
         [ 1.2856e+01,  1.6769e+01, -2.1377e+01],
         [ 3.4195e+01, -3.2903e+00, -2.9654e+01],
         [-3.1014e+01,  1.6444e+01,  6.4842e+01],
         [ 1.4955e+00, -1.4509e+00, -2.1269e+01]

## Dynamics

For performing the dynamics we will create an `Integrator` object for integrating the time steps of the simulation as well as a `Wrapper` object for wrapping the system coordinates within the periodic cell

In [ ]:
from torchmd.integrator import Integrator
from torchmd.wrapper import Wrapper

langevin_temperature = 300  # K
langevin_gamma = 0.1
timestep = 1  # fs

integrator = Integrator(system, forces, timestep, device, gamma=langevin_gamma, T=langevin_temperature)
wrapper = Wrapper(mol.numAtoms, mol.bonds if len(mol.bonds) else None, device)

In [ ]:
from torchmd.minimizers import minimize_bfgs

minimize_bfgs(system, forces, steps=500)  # Minimize the system

Create a CSV file logger for the simulation which keeps track of the energies and temperature.

In [ ]:
from torchmd.utils import LogWriter

logger = LogWriter(path="logs/", keys=('iter','ns','epot','ekin','etot','T'), name='monitor.csv')

Now we can finally perform the full dynamics

In [ ]:
from tqdm import tqdm 
import numpy as np

FS2NS = 1E-6 # Femtosecond to nanosecond conversion

steps = 1000
output_period = 10
save_period = 100
traj = []

trajectoryout = "mytrajectory.npy"

iterator = tqdm(range(1, int(steps / output_period) + 1))
Epot = forces.compute(system.pos, system.box, system.forces)
for i in iterator:
    Ekin, Epot, T = integrator.step(niter=output_period)
    wrapper.wrap(system.pos, system.box)
    currpos = system.pos.detach().cpu().numpy().copy()
    traj.append(currpos)
    
    if (i*output_period) % save_period  == 0:
        np.save(trajectoryout, np.stack(traj, axis=2))

    logger.write_row({'iter':i*output_period,'ns':FS2NS*i*output_period*timestep,'epot':Epot,'ekin':Ekin,'etot':Epot+Ekin,'T':T})